# Download selected variables from the INGRID CMIP6 database
* Organize data by variable/model and save each individual year into a separate file. Easiest way, I think, to more uniformly handle things because of loack of standardization in datafiles.
* Database here: http://mary.ldeo.columbia.edu:81/CMIP6/
* For master list of everything available: wget http://mary.ldeo.columbia.edu/master_collection/mary_cmip6.csv

## Start Clean

In [11]:
# Reset the environment (start clean)
%reset -f

# Import Modules and define functions
import calendar
import datetime
import os
import numpy as np
import numpy.ma as ma
import netCDF4
import matplotlib
import copy
from matplotlib import pyplot as plt
import scipy
import scipy.signal
#import scipy.io as sio
#import seaborn as sns
#import pandas as pd
import scipy.stats as stats
#import statsmodels.api as sm
from IPython.display import display
import requests
import pandas as pd
import os

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


In [31]:
all_models=False
if all_models:
# Model Names
    model_names=(    'AWI-CM-1-1-MR',	
                     'BCC-CSM2-MR',		
                     'BCC-ESM1',
                     'CAMS-CSM1-0',      # NEED TO REDO
                     'CanESM5',			
                     'CESM2',			
                     'CESM2-WACCM',		
                     'CNRM-CM6-1',		
                     'CNRM-ESM2-1',		
                     'E3SM-1-0',		
                     'EC-Earth3',       # NEED TO REDO
                     'EC-Earth3-LR',    # NEED TO REDO
                     'EC-Earth3-Veg',   # NEED TO REDO
                     'FGOALS-f3-L',     # NEED TO REDO
                     'GFDL-AM4', 
                     'GFDL-CM4',  # PI CONTROL PROBLEMS		
                     'GFDL-ESM4',		
                     'GISS-E2-1-G',		
                     'GISS-E2-1-H',		
                     'HadGEM3-GC31-LL',
                     'IPSL-CM6A-LR',	 
                     'MIROC6',			# mrsos and tas done
                     'MIROC-ES2L',
                     'MPI-ESM1-2-HR',   
                     'MRI-ESM2-0',       # mrsos and tas done
                     'NESM3',             # mrsos and tas done
                     'SAM0-UNICON',		# mrsos and tas done
                     'UKESM1-0-LL',	    # mrsos and tas done
                  )
else:
    model_names=['IPSL-CM6A-LR']
for x in enumerate(model_names):
    print(x)

(0, 'IPSL-CM6A-LR')


## Setup: Choose models/ensemble members/variables/etc

In [14]:
# Baseline directory
base_dir   = 'http://mary.ldeo.columbia.edu:81/CMIP6/.'

# Output directory
out_dir = '/Users/kmarvel/Documents/DATA/LGM/FromMary/'

# MIP/Sim Name (WITH YEAR RANGES)
experiment_id = ('piControl')   # historical
#experiment_id = ('historical')   # historical
#experiment_id = ('ssp245')       # RCP 4.5 equivalent
#experiment_id = ('ssp585')       # RCP 8.5 equivalent
#experiment_id = ('ssp370')       # RCP 7.0 equivalent
    
# Variable info: name, category, units_old, units_new, scale for new unites 
var_info = np.array([  ['rsdt','W m-2', 'W m-2',1, 'TOA Incident Shortwave Radiation'],    
                       ['rsut','W m-2', 'W m-2',1, 'TOA Outgoing Shortwave Radiation'],
                       ['rsutcs','W m-2', 'W m-2',1, 'TOA Clear-sky Outgoing Shortwave Radiation'],
                       ['rlut','W m-2', 'W m-2',1, 'TOA Outgoing Longwave Radiation'],
                       ['rlutcs','W m-2', 'W m-2',1, 'TOA Clear-sky Outgoing Longwave Radiation'],
                       ['ts','K','K',1,'surface air temperature'],
            ])

#var_info = np.array([   ['mrsos','kg m-2','kg m-2',1,'surface (top 10cm) soil moisture content (all phases)'],  
#            ])

#var_info = np.array([  ['evspsbl', 'kg m-2 s-1', 'mm day-1',86400, 'evaporation rate, including transpiration and sublimation'],
#
#            ])

# Load dataframe containing all information available
ingrid_cmip6 = pd.read_csv(out_dir+'mary_cmip6_101419.csv')
#ingrid_cmip6 = pd.read_csv('../filelists/mary_cmip6_092319.csv')
#ingrid_cmip6 = pd.read_csv('../filelists/mary_cmip6_082619.csv')
#ingrid_cmip6 = pd.read_csv('../filelists/mary_cmip6_073119.csv')
#ingrid_cmip6 = pd.read_csv('../filelists/mary_cmip6_073019.csv')
print(var_info)
ingrid_cmip6

# Initialize master list to use for post-processing
df_proclist = pd.DataFrame(columns=['model','sim','ensemble','variable'])

# just some debugging and testing
#df1 = ingrid_cmip6[(ingrid_cmip6.source_id==curr_mod) & (ingrid_cmip6.variable_id==curr_var) & (ingrid_cmip6.experiment_id == experiment_id)]
#df1 = ingrid_cmip6[(ingrid_cmip6.source_id=='BCC-CSM2-MR') & (ingrid_cmip6.variable_id=='lai')]
#df1
df_proclist

[['rsdt' 'W m-2' 'W m-2' '1' 'TOA Incident Shortwave Radiation']
 ['rsut' 'W m-2' 'W m-2' '1' 'TOA Outgoing Shortwave Radiation']
 ['rsutcs' 'W m-2' 'W m-2' '1'
  'TOA Clear-sky Outgoing Shortwave Radiation']
 ['rlut' 'W m-2' 'W m-2' '1' 'TOA Outgoing Longwave Radiation']
 ['rlutcs' 'W m-2' 'W m-2' '1'
  'TOA Clear-sky Outgoing Longwave Radiation']
 ['ts' 'K' 'K' '1' 'surface air temperature']]


,model,sim,ensemble,variable


## For Accessing Directory CMIP6 (NOT CMIP6i)
* Grab everything available. Put the years in the filenames.
* also grabbed fixed variables, if available: sftlf, areacella

In [34]:
# Nested model/variable loop
for n_mod in enumerate(model_names):

    # Variable/model Info
    curr_mod     = n_mod[1]
    
    print(curr_mod)
#     # LOAD GRIDCELL AREA, IF AVAILABLE (units, m2)----------------------------------------------------------------------------------------------------------------------------------
#     df_areacell       = ingrid_cmip6[(ingrid_cmip6.source_id==curr_mod) & (ingrid_cmip6.variable_id=='areacella') & (ingrid_cmip6.experiment_id == experiment_id)]

#     if df_areacell.empty==False:
#         areacell_link = base_dir+df_areacell.activity_id.iloc[0]+'/.'+df_areacell.institution_id.iloc[0]+'/.'+curr_mod+'/.'+experiment_id+'/.'+df_areacell.member_id.iloc[0]+'/.'+df_areacell.table_id.iloc[0]+ \
#                             '/.areacella/.'+df_areacell.grid_label.iloc[0]+'/.'+df_areacell.version.iloc[0]+'/.'+df_areacell .file_basename.iloc[0]+'/.areacella/dods'
    
#         # Create a Remote Link
#         nc_file = netCDF4.Dataset(areacell_link) 
    
#         # Pull out variables 
#         areacell = nc_file.variables['areacella'][:]
#         lat      = nc_file.variables['lat'][:]
#         lon      = nc_file.variables['lon'][:]
    
#         # Close
#         nc_file.close
        
#     # LOAD LAND FRACTION OF GRID CELL, IF AVAILABLE (percent)--------------------------------------------------------------------------------------------------------------------
#     df_landfrac       = ingrid_cmip6[(ingrid_cmip6.source_id==curr_mod) & (ingrid_cmip6.variable_id=='sftlf') & (ingrid_cmip6.experiment_id == experiment_id)]

#     if df_landfrac.empty==False:
#         landfrac_link     = base_dir+df_landfrac.activity_id.iloc[0]+'/.'+df_landfrac.institution_id.iloc[0]+'/.'+curr_mod+'/.'+experiment_id+'/.'+df_landfrac.member_id.iloc[0]+'/.'+df_landfrac.table_id.iloc[0]+ \
#                         '/.sftlf/.'+df_landfrac.grid_label.iloc[0]+'/.'+df_landfrac.version.iloc[0]+'/.'+df_landfrac.file_basename.iloc[0]+'/.sftlf/dods'

#         # Create a Remote Link
#         nc_file = netCDF4.Dataset(landfrac_link) 
    
#         # Pull out variables 
#         landfrac = nc_file.variables['sftlf'][:]
#         lat      = nc_file.variables['lat'][:]
#         lon      = nc_file.variables['lon'][:]
    
#         # Close
#         nc_file.close 
#     else:
#         landfrac=np.array([0,0])
        
    # Variables Loop--------------------------------------------------------------------------------------------------------------------------------------------------------------
    for n_var in enumerate(var_info):
        
        curr_var     = n_var[1][0]
        print(curr_var)
        curr_units   = n_var[1][2]
        curr_scale   = np.float(n_var[1][3])
        curr_name    = n_var[1][4]
    
        # search for all versions for one model
        df1 = ingrid_cmip6[(ingrid_cmip6.source_id==curr_mod) & (ingrid_cmip6.variable_id==curr_var) & (ingrid_cmip6.experiment_id == experiment_id)]

        # OLD Drop duplicate ensemble members (split by year)
        #df1.drop_duplicates(subset ="member_id",keep = False, inplace = True) 

        # Number of ensemble members available
        num_ens=df1.shape[0]
        print("number of ensembles = "+str(num_ens))
        #fig = plt.figure(figsize=(12, 7),facecolor="white")
        #plt.pcolormesh(lon,lat,areacell),plt.colorbar()
        #fig = plt.figure(figsize=(12, 7),facecolor="white")
        #plt.pcolormesh(lon,lat,landfrac),plt.colorbar()
        #fig = plt.figure(figsize=(12, 7),facecolor="white")
        #plt.pcolormesh(lon,lat,(landfrac/100)*areacell),plt.colorbar()
        
        # Ensemble Loop----------------------------------------------------------------------------------------------------------------------------------------------------------
        for i_ens in np.arange(num_ens):

            # Construct Remote OpenDAP Link
            nc_link = base_dir+df1.activity_id.iloc[i_ens]+'/.'+df1.institution_id.iloc[i_ens]+'/.'+curr_mod+'/.'+experiment_id+'/.'+df1.member_id.iloc[i_ens]+'/.'+df1.table_id.iloc[i_ens]+ \
                        '/.'+curr_var+'/.'+df1.grid_label.iloc[i_ens]+'/.'+df1.version.iloc[i_ens]+'/.'+df1.file_basename.iloc[i_ens]+'/.'+curr_var+'/dods'

            print(curr_mod+':'+experiment_id+':'+df1.member_id.iloc[i_ens]+':'+curr_var)
            df_proclist.append(pd.Series([curr_mod,experiment_id,df1.member_id.iloc[i_ens],curr_var],index=df_proclist.columns),ignore_index=True)
            
            # Check if link exists
            request = requests.get(nc_link)    
            
            # If this Exists, then load the data
            if request.status_code==200:
                #print(nc_link)       
        
                # Open the remote file-----------------------------------------------------------------------------------------------------------------------------------------

                # Create a Remote Link
                nc_file = netCDF4.Dataset(nc_link) 
    
                # Pull out variables 
                var_nc = nc_file.variables[curr_var]
                lat    = nc_file.variables['lat'][:]
                lon    = nc_file.variables['lon'][:]

                # Time Stuff
                nctime    = nc_file.variables['time'][:]          # time data
                unit_time = nc_file.variables['time'].units       # units
                cal_time  = nc_file.variables['time'].calendar    # calendar 

                # Loop through and pull out year and month vectors from datetime object. 
                # I will use these for indexing the 20th Century Reanalysis Data
                yr_vect  = np.zeros(np.shape(nctime))
                mon_vect = np.zeros(np.shape(nctime))

                for i_date in enumerate(nctime):
                    yr_vect[i_date[0]]  = netCDF4.num2date(nctime[i_date[0]],units=unit_time,calendar=cal_time).year
                    mon_vect[i_date[0]] = netCDF4.num2date(nctime[i_date[0]],units=unit_time,calendar=cal_time).month

                # Year Vector (one value per year)
                yrs     = np.unique(yr_vect)

                # -------------------------------------------------------------------------------------------------------------------------------------------------------------
                # Pull out the data year by year and save in individual year files
                
                # Make model directory if it does not exist
                if os.path.isdir(out_dir+'/'+curr_var+'/'+curr_mod)==False:
                    os.makedirs(out_dir+'/'+curr_var+'/'+curr_mod)
                
                # Year Loop
                for i_yr in enumerate(yrs):
                    # Locate all months, current year
                    loc_yr = np.where(yr_vect==i_yr[1])[0]
    
                    # Pull out current year of data and put in the storage array
                    var_data = copy.deepcopy(var_nc[loc_yr,:,:])
    
                    # Scale the data
                    var_data = var_data*curr_scale

                    # Save current year of data to netcdf file
   
                    # Output name
                    fname_out = out_dir+'/'+curr_var+'/'+curr_mod+'/'+curr_var+'.'+experiment_id+'.'+curr_mod+'.'+df1.member_id.iloc[i_ens]+'.' \
                            +np.str(np.int(i_yr[1]))+'.nc'
      
                    # If this file exists, delete it and overwrite
                    if os.path.isfile(fname_out)==True:
                        os.remove(fname_out)
    
                    # Create and open output file
                    print("WRITING "+fname_out)
                    ncout = netCDF4.Dataset(fname_out,'w',clobber=True,format="NETCDF4_CLASSIC")
                    
                    # Create Dimensions
                    ncout.createDimension('lat',  np.size(lat))
                    ncout.createDimension('lon',  np.size(lon))
                    ncout.createDimension('mon',  np.size(mons))

                    # Create Variables (info: name, precision, dimensions)
                    # dimensions
                    lat_nc   = ncout.createVariable('lat', float, ('lat'), zlib=True)
                    lon_nc   = ncout.createVariable('lon', float, ('lon'), zlib=True)
                    mon_nc   = ncout.createVariable('mon', float, ('mon'), zlib=True)
    
                    # Two-Dimensional Variables
                    varout_nc  = ncout.createVariable(curr_var, float, ('mon','lat','lon'), zlib=True)

                    # Write out dimensional data
                    lat_nc[:]  = lat;         lat_nc.long_name  = 'latitude';  lat_nc.units = 'degrees_north';
                    lon_nc[:]  = lon;         lon_nc.long_name  = 'longitude'; lon_nc.units = 'degrees_east';
                    mon_nc[:]  = mons;        mon_nc.long_name  = 'month';     mon_nc.units = 'unitless';   

                    # Write out data
                    varout_nc[:] = var_data; varout_nc.long_name = curr_name;  varout_nc.units = curr_units;

#                     # If time invariant information is available
#                     if df_areacell.empty==False:
#                         areacell_nc    = ncout.createVariable('areacell', float, ('lat','lon'), zlib=True)
#                         areacell_nc[:] = areacell; areacell_nc.long_name = 'grid cell area for atmospheric grid variables';  areacell_nc.units = 'm2';

#                     # For some reason GFDL land mask not the same resolution. Just ignore for now.
#                     #if df_landfrac.empty==False:
#                     if ((df_landfrac.empty==False) & (landfrac.shape[0]==lat.size)):
#                         landfrac_nc    = ncout.createVariable('landfrac', float, ('lat','lon'), zlib=True)
#                         landfrac_nc[:] = landfrac; landfrac_nc.long_name = 'percentage of the grid cell occupied by land (including lakes)';  landfrac_nc.units = 'percent';
            
                    # File Information
                    ncout.comment = 'CMIP6, '+experiment_id+' run. '+curr_mod+': '+df1.member_id.iloc[i_ens]

                # Close the file
                ncout.close()

IPSL-CM6A-LR
rsdt
number of ensembles = 0
rsut
number of ensembles = 5
IPSL-CM6A-LR:piControl:r1i1p1f1:rsut


ConnectionError: HTTPConnectionPool(host='mary.ldeo.columbia.edu', port=81): Max retries exceeded with url: /CMIP6/.CMIP/.IPSL/.IPSL-CM6A-LR/.piControl/.r1i1p1f1/.Amon/.rsut/.gr/.v20180802/.rsut_Amon_IPSL-CM6A-LR_piControl_r1i1p1f1_gr_185001-234912.nc/.rsut/dods (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7ffb100228d0>: Failed to establish a new connection: [Errno 60] Operation timed out',))

In [23]:
curr_mod='IPSL-CM6A-LR'
curr_var="rsut"
df1 = ingrid_cmip6[(ingrid_cmip6.source_id==curr_mod) & (ingrid_cmip6.variable_id==curr_var) & (ingrid_cmip6.experiment_id == experiment_id)]
df1

,activity_id,experiment_id,file_basename,file_dirname,file_fullpath,grid_label,institution_id,member_id,mip_era,source_id,table_id,time_range,variable_id,tracking_id,version
748,CMIP,piControl,rsut_Amon_IPSL-CM6A-LR_piControl_r1i1p1f1_gr_1...,/m2/haibo/CMIP6mon/CMIP/IPSL/IPSL-CM6A-LR/piCo...,/m2/haibo/CMIP6mon/CMIP/IPSL/IPSL-CM6A-LR/piCo...,gr,IPSL,r1i1p1f1,CMIP6,IPSL-CM6A-LR,Amon,185001-234912,rsut,NaN,v20180802
90693,CMIP,piControl,rsut_Amon_IPSL-CM6A-LR_piControl_r1i1p1f1_gr_2...,/m5/haibo/CMIP6mon/CMIP/IPSL/IPSL-CM6A-LR/piCo...,/m5/haibo/CMIP6mon/CMIP/IPSL/IPSL-CM6A-LR/piCo...,gr,IPSL,r1i1p1f1,CMIP6,IPSL-CM6A-LR,Amon,285001-304912,rsut,NaN,v20181123
90695,CMIP,piControl,rsut_Amon_IPSL-CM6A-LR_piControl_r1i1p1f1_gr_2...,/m5/haibo/CMIP6mon/CMIP/IPSL/IPSL-CM6A-LR/piCo...,/m5/haibo/CMIP6mon/CMIP/IPSL/IPSL-CM6A-LR/piCo...,gr,IPSL,r1i1p1f1,CMIP6,IPSL-CM6A-LR,Amon,235001-284912,rsut,NaN,v20181123
98654,CMIP,piControl,rsut_Amon_IPSL-CM6A-LR_piControl_r1i2p1f1_gr_2...,/m5/haibo/CMIP6mon/CMIP/IPSL/IPSL-CM6A-LR/piCo...,/m5/haibo/CMIP6mon/CMIP/IPSL/IPSL-CM6A-LR/piCo...,gr,IPSL,r1i2p1f1,CMIP6,IPSL-CM6A-LR,Amon,209901-209912,rsut,NaN,v20190319
98655,CMIP,piControl,rsut_Amon_IPSL-CM6A-LR_piControl_r1i2p1f1_gr_1...,/m5/haibo/CMIP6mon/CMIP/IPSL/IPSL-CM6A-LR/piCo...,/m5/haibo/CMIP6mon/CMIP/IPSL/IPSL-CM6A-LR/piCo...,gr,IPSL,r1i2p1f1,CMIP6,IPSL-CM6A-LR,Amon,185001-209812,rsut,NaN,v20190319


In [17]:
#df1 = ingrid_cmip6[(ingrid_cmip6.source_id==curr_mod) & (ingrid_cmip6.variable_id==curr_var)]
df1 = ingrid_cmip6[(ingrid_cmip6.source_id==curr_mod) & (ingrid_cmip6.variable_id=='rsut')]

df1

,activity_id,experiment_id,file_basename,file_dirname,file_fullpath,grid_label,institution_id,member_id,mip_era,source_id,table_id,time_range,variable_id,tracking_id,version
532,CMIP,abrupt-4xCO2,rsut_Amon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_g...,/m2/haibo/CMIP6mon/CMIP/IPSL/IPSL-CM6A-LR/abru...,/m2/haibo/CMIP6mon/CMIP/IPSL/IPSL-CM6A-LR/abru...,gr,IPSL,r1i1p1f1,CMIP6,IPSL-CM6A-LR,Amon,185001-214912,rsut,NaN,v20180727
574,CMIP,1pctCO2,rsut_Amon_IPSL-CM6A-LR_1pctCO2_r1i1p1f1_gr_185...,/m2/haibo/CMIP6mon/CMIP/IPSL/IPSL-CM6A-LR/1pct...,/m2/haibo/CMIP6mon/CMIP/IPSL/IPSL-CM6A-LR/1pct...,gr,IPSL,r1i1p1f1,CMIP6,IPSL-CM6A-LR,Amon,185001-199912,rsut,NaN,v20180727
748,CMIP,piControl,rsut_Amon_IPSL-CM6A-LR_piControl_r1i1p1f1_gr_1...,/m2/haibo/CMIP6mon/CMIP/IPSL/IPSL-CM6A-LR/piCo...,/m2/haibo/CMIP6mon/CMIP/IPSL/IPSL-CM6A-LR/piCo...,gr,IPSL,r1i1p1f1,CMIP6,IPSL-CM6A-LR,Amon,185001-234912,rsut,NaN,v20180802
51157,CMIP,historical,rsut_Amon_IPSL-CM6A-LR_historical_r32i1p1f1_gr...,/m3/haibo/CMIP6mon/CMIP/IPSL/IPSL-CM6A-LR/hist...,/m3/haibo/CMIP6mon/CMIP/IPSL/IPSL-CM6A-LR/hist...,gr,IPSL,r32i1p1f1,CMIP6,IPSL-CM6A-LR,Amon,185001-201412,rsut,NaN,v20190802
55415,DCPP,dcppC-ipv-NexTrop-pos,rsut_Amon_IPSL-CM6A-LR_dcppC-ipv-NexTrop-pos_r...,/m4/haibo/CMIP6mon/DCPP/IPSL/IPSL-CM6A-LR/dcpp...,/m4/haibo/CMIP6mon/DCPP/IPSL/IPSL-CM6A-LR/dcpp...,gr,IPSL,r8i1p1f1,CMIP6,IPSL-CM6A-LR,Amon,185001-185912,rsut,NaN,v20190110
55457,DCPP,dcppC-ipv-NexTrop-pos,rsut_Amon_IPSL-CM6A-LR_dcppC-ipv-NexTrop-pos_r...,/m4/haibo/CMIP6mon/DCPP/IPSL/IPSL-CM6A-LR/dcpp...,/m4/haibo/CMIP6mon/DCPP/IPSL/IPSL-CM6A-LR/dcpp...,gr,IPSL,r1i1p1f1,CMIP6,IPSL-CM6A-LR,Amon,185001-185912,rsut,NaN,v20190110
55479,DCPP,dcppC-amv-ExTrop-neg,rsut_Amon_IPSL-CM6A-LR_dcppC-amv-ExTrop-neg_r1...,/m4/haibo/CMIP6mon/DCPP/IPSL/IPSL-CM6A-LR/dcpp...,/m4/haibo/CMIP6mon/DCPP/IPSL/IPSL-CM6A-LR/dcpp...,gr,IPSL,r17i1p1f1,CMIP6,IPSL-CM6A-LR,Amon,185001-185912,rsut,NaN,v20190110
55543,DCPP,dcppC-ipv-NexTrop-pos,rsut_Amon_IPSL-CM6A-LR_dcppC-ipv-NexTrop-pos_r...,/m4/haibo/CMIP6mon/DCPP/IPSL/IPSL-CM6A-LR/dcpp...,/m4/haibo/CMIP6mon/DCPP/IPSL/IPSL-CM6A-LR/dcpp...,gr,IPSL,r6i1p1f1,CMIP6,IPSL-CM6A-LR,Amon,185001-185912,rsut,NaN,v20190110
55575,DCPP,dcppC-ipv-NexTrop-pos,rsut_Amon_IPSL-CM6A-LR_dcppC-ipv-NexTrop-pos_r...,/m4/haibo/CMIP6mon/DCPP/IPSL/IPSL-CM6A-LR/dcpp...,/m4/haibo/CMIP6mon/DCPP/IPSL/IPSL-CM6A-LR/dcpp...,gr,IPSL,r9i1p1f1,CMIP6,IPSL-CM6A-LR,Amon,185001-185912,rsut,NaN,v20190110
55645,DCPP,dcppC-ipv-NexTrop-pos,rsut_Amon_IPSL-CM6A-LR_dcppC-ipv-NexTrop-pos_r...,/m4/haibo/CMIP6mon/DCPP/IPSL/IPSL-CM6A-LR/dcpp...,/m4/haibo/CMIP6mon/DCPP/IPSL/IPSL-CM6A-LR/dcpp...,gr,IPSL,r7i1p1f1,CMIP6,IPSL-CM6A-LR,Amon,185001-185912,rsut,NaN,v20190110


In [27]:
curr_mod

'MPI-ESM1-2-HR'

In [21]:
ingrid_cmip6

,activity_id,experiment_id,file_basename,file_dirname,file_fullpath,grid_label,institution_id,member_id,mip_era,source_id,table_id,time_range,variable_id,tracking_id,version
0,CMIP,historical,va_Amon_GFDL-CM4_historical_r1i1p1f1_gr1_18500...,/m3/haibo/CMIP6mon/CMIP/NOAA-GFDL/GFDL-CM4/his...,/m3/haibo/CMIP6mon/CMIP/NOAA-GFDL/GFDL-CM4/his...,gr1,NOAA-GFDL,r1i1p1f1,CMIP6,GFDL-CM4,Amon,185001-185412,va,NaN,v20180301
1,CMIP,historical,vas_Amon_GFDL-CM4_historical_r1i1p1f1_gr1_1850...,/m3/haibo/CMIP6mon/CMIP/NOAA-GFDL/GFDL-CM4/his...,/m3/haibo/CMIP6mon/CMIP/NOAA-GFDL/GFDL-CM4/his...,gr1,NOAA-GFDL,r1i1p1f1,CMIP6,GFDL-CM4,Amon,185001-185412,vas,NaN,v20180301
2,CMIP,historical,ta_Amon_GFDL-CM4_historical_r1i1p1f1_gr1_18500...,/m3/haibo/CMIP6mon/CMIP/NOAA-GFDL/GFDL-CM4/his...,/m3/haibo/CMIP6mon/CMIP/NOAA-GFDL/GFDL-CM4/his...,gr1,NOAA-GFDL,r1i1p1f1,CMIP6,GFDL-CM4,Amon,185001-185412,ta,NaN,v20180301
3,CMIP,historical,zg_Amon_GFDL-CM4_historical_r1i1p1f1_gr1_18500...,/m3/haibo/CMIP6mon/CMIP/NOAA-GFDL/GFDL-CM4/his...,/m3/haibo/CMIP6mon/CMIP/NOAA-GFDL/GFDL-CM4/his...,gr1,NOAA-GFDL,r1i1p1f1,CMIP6,GFDL-CM4,Amon,185001-185412,zg,NaN,v20180301
4,CMIP,historical,sfcWind_Amon_GFDL-CM4_historical_r1i1p1f1_gr1_...,/m3/haibo/CMIP6mon/CMIP/NOAA-GFDL/GFDL-CM4/his...,/m3/haibo/CMIP6mon/CMIP/NOAA-GFDL/GFDL-CM4/his...,gr1,NOAA-GFDL,r1i1p1f1,CMIP6,GFDL-CM4,Amon,185001-185412,sfcWind,NaN,v20180301
5,CMIP,historical,wap_Amon_GFDL-CM4_historical_r1i1p1f1_gr1_1850...,/m3/haibo/CMIP6mon/CMIP/NOAA-GFDL/GFDL-CM4/his...,/m3/haibo/CMIP6mon/CMIP/NOAA-GFDL/GFDL-CM4/his...,gr1,NOAA-GFDL,r1i1p1f1,CMIP6,GFDL-CM4,Amon,185001-185412,wap,NaN,v20180301
6,CMIP,historical,pr_Amon_GFDL-CM4_historical_r1i1p1f1_gr1_18500...,/m3/haibo/CMIP6mon/CMIP/NOAA-GFDL/GFDL-CM4/his...,/m3/haibo/CMIP6mon/CMIP/NOAA-GFDL/GFDL-CM4/his...,gr1,NOAA-GFDL,r1i1p1f1,CMIP6,GFDL-CM4,Amon,185001-185412,pr,NaN,v20180301
7,CMIP,historical,rsds_Amon_GFDL-CM4_historical_r1i1p1f1_gr1_185...,/m3/haibo/CMIP6mon/CMIP/NOAA-GFDL/GFDL-CM4/his...,/m3/haibo/CMIP6mon/CMIP/NOAA-GFDL/GFDL-CM4/his...,gr1,NOAA-GFDL,r1i1p1f1,CMIP6,GFDL-CM4,Amon,185001-185412,rsds,NaN,v20180301
8,CMIP,historical,uas_Amon_GFDL-CM4_historical_r1i1p1f1_gr1_1850...,/m3/haibo/CMIP6mon/CMIP/NOAA-GFDL/GFDL-CM4/his...,/m3/haibo/CMIP6mon/CMIP/NOAA-GFDL/GFDL-CM4/his...,gr1,NOAA-GFDL,r1i1p1f1,CMIP6,GFDL-CM4,Amon,185001-185412,uas,NaN,v20180301
9,CMIP,historical,prsn_Amon_GFDL-CM4_historical_r1i1p1f1_gr1_185...,/m3/haibo/CMIP6mon/CMIP/NOAA-GFDL/GFDL-CM4/his...,/m3/haibo/CMIP6mon/CMIP/NOAA-GFDL/GFDL-CM4/his...,gr1,NOAA-GFDL,r1i1p1f1,CMIP6,GFDL-CM4,Amon,185001-185412,prsn,NaN,v20180301


## Save dataframe containing all file information

In [71]:
nc_link
df1.activity_id.iloc[0]

'ScenarioMIP'

In [74]:
nc_link

'http://mary.ldeo.columbia.edu:81/CMIP6/.CMIP/.ScenarioMIP/.BCC/.BCC-CSM2-MR/.ssp245/.r1i1p1f1/.Lmon/.mrsos/.gn/.v20190308/.mrsos_Lmon_BCC-CSM2-MR_ssp245_r1i1p1f1_gn_201501-210012.nc/.mrsos/dods'

In [69]:
http://mary.ldeo.columbia.edu:81/CMIP6/.ScenarioMIP/.BCC/.BCC-CSM2-MR/.ssp245/.r1i1p1f1/.Lmon/.mrsos/.gn/.v20190308/.mrsos_Lmon_BCC-CSM2-MR_ssp245_r1i1p1f1_gn_201501-210012.nc/.mrsos/dods 

0

In [60]:
requests.get(nc_link)  

<Response [404]>

In [61]:
df1

,activity_id,experiment_id,file_basename,file_dirname,file_fullpath,grid_label,institution_id,member_id,mip_era,source_id,table_id,time_range,variable_id,tracking_id,version
74647,ScenarioMIP,ssp245,mrsos_Lmon_BCC-CSM2-MR_ssp245_r1i1p1f1_gn_2015...,/m5/haibo/CMIP6mon/ScenarioMIP/BCC/BCC-CSM2-MR...,/m5/haibo/CMIP6mon/ScenarioMIP/BCC/BCC-CSM2-MR...,gn,BCC,r1i1p1f1,CMIP6,BCC-CSM2-MR,Lmon,201501-210012,mrsos,NaN,v20190308


KeyError: 0